<a href="https://colab.research.google.com/github/Praveenkumarbalaji/Praveen/blob/main/DocuLens_AI_Intelligent_Legal_Document_Analysis_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install pdfplumber

import pdfplumber

pdf_path = "/content/google_terms_of_service_en_in.pdf"

output_text_file = "extracted_text.txt"

with pdfplumber.open(pdf_path) as pdf:
    extracted_text = ""
    for page in pdf.pages:
        extracted_text += page.extract_text()

with open(output_text_file, "w") as text_file:
    text_file.write(extracted_text)

print(f"Text extracted and saved to {output_text_file}")

Text extracted and saved to extracted_text.txt


In [29]:
# reading pdf content
with open("/content/extracted_text.txt", "r") as file:
    document_text = file.read()

# preview the document content
print(document_text[:500])  # preview the first 500 characters

GOOGLE TERMS OF SERVICE
Effective May 22, 2024 | Archived versions
What’s covered in these terms
We know it’s tempting to skip these Terms of
Service, but it’s important to establish what you
can expect from us as you use Google services,
and what we expect from you.
These Terms of Service re ect the way Google’s business works, the laws that apply to
our company, and certain things we’ve always believed to be true. As a result, these Terms
of Service help de ne Google’s relationship with you as


In [30]:
from transformers import pipeline

# load the summarization pipeline
summarizer = pipeline("summarization", model="t5-small")

# summarize the document text (you can summarize parts if the document is too large)
summary = summarizer(document_text[:1000], max_length=150, min_length=30, do_sample=False)
print("Summary:", summary[0]['summary_text'])

Device set to use cpu


Summary: these Terms of Service reect the way Google’s business works, the laws that apply to our company, and certain things we’ve always believed to be true . these terms include: what you can expect from us, which describes how we provide and develop our services What we expect from you, which establishes certain rules for using our services Content in Google services .


In [37]:
import nltk
import os

# Let's check what files actually exist in the tokenizers directory
nltk_data_dir = '/home/jovyan/nltk_data/tokenizers'
print("Contents of tokenizers directory:")
if os.path.exists(nltk_data_dir):
    print(os.listdir(nltk_data_dir))
else:
    print("Directory does not exist")

Contents of tokenizers directory:
['punkt', 'punkt.zip']


In [41]:
import pdfplumber
import nltk
from nltk.tokenize import sent_tokenize

# Extract text from PDF
with pdfplumber.open('google_terms_of_service_en_in.pdf') as pdf:
    document_text = ""
    for page in pdf.pages:
        document_text += page.extract_text()

# Process the text in smaller chunks to avoid any memory issues
chunk_size = 5000  # Process 5000 characters at a time
chunks = [document_text[i:i+chunk_size] for i in range(0, len(document_text), chunk_size)]

all_sentences = []
for chunk in chunks:
    try:
        sentences = sent_tokenize(chunk)
        all_sentences.extend(sentences)
    except Exception as e:
        print(f"Error processing chunk: {str(e)}")
        continue

# Combine sentences into passages
passages = []
current_passage = ""

for sentence in all_sentences:
    if len(current_passage.split()) + len(sentence.split()) < 200:
        current_passage += " " + sentence if current_passage else sentence
    else:
        passages.append(current_passage.strip())
        current_passage = sentence

if current_passage:
    passages.append(current_passage.strip())

print(f"Successfully processed {len(all_sentences)} sentences into {len(passages)} passages")
print("\
First passage preview:")
print(passages[0] if passages else "No passages created")

Error processing chunk: 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt_tab/english/

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/usr/local/nltk_data'
    - '/usr/local/nltk_data'
    - '/root/nltk_data'
    - '/root/nltk_data'
    - '/root/nltk_data'
    - '/usr/local/nltk_data'
    - '/home/jovyan/nltk_data'
    - '/home/jovyan/nltk_data'
**********************************************************************

Error processing chunk: 
**********************************************************************
  Resource punkt_t

In [42]:
# load the question generation pipeline
qg_pipeline = pipeline("text2text-generation", model="valhalla/t5-base-qg-hl")

# function to generate questions using the pipeline
def generate_questions_pipeline(passage, min_questions=3):
    input_text = f"generate questions: {passage}"
    results = qg_pipeline(input_text)
    questions = results[0]['generated_text'].split('<sep>')

    # ensure we have at least 3 questions
    questions = [q.strip() for q in questions if q.strip()]

    # if fewer than 3 questions, try to regenerate from smaller parts of the passage
    if len(questions) < min_questions:
        passage_sentences = passage.split('. ')
        for i in range(len(passage_sentences)):
            if len(questions) >= min_questions:
                break
            additional_input = ' '.join(passage_sentences[i:i+2])
            additional_results = qg_pipeline(f"generate questions: {additional_input}")
            additional_questions = additional_results[0]['generated_text'].split('<sep>')
            questions.extend([q.strip() for q in additional_questions if q.strip()])

    return questions[:min_questions]  # return only the top 3 questions

# generate questions from passages
for idx, passage in enumerate(passages):
    questions = generate_questions_pipeline(passage)
    print(f"Passage {idx+1}:\n{passage}\n")
    print("Generated Questions:")
    for q in questions:
        print(f"- {q}")
    print(f"\n{'-'*50}\n")

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/15.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Device set to use cpu


In [43]:
# load the question generation pipeline
qg_pipeline = pipeline("text2text-generation", model="valhalla/t5-base-qg-hl")

# function to generate questions using the pipeline
def generate_questions_pipeline(passage, min_questions=3):
    input_text = f"generate questions: {passage}"
    results = qg_pipeline(input_text)
    questions = results[0]['generated_text'].split('<sep>')

    # ensure we have at least 3 questions
    questions = [q.strip() for q in questions if q.strip()]

    # if fewer than 3 questions, try to regenerate from smaller parts of the passage
    if len(questions) < min_questions:
        passage_sentences = passage.split('. ')
        for i in range(len(passage_sentences)):
            if len(questions) >= min_questions:
                break
            additional_input = ' '.join(passage_sentences[i:i+2])
            additional_results = qg_pipeline(f"generate questions: {additional_input}")
            additional_questions = additional_results[0]['generated_text'].split('<sep>')
            questions.extend([q.strip() for q in additional_questions if q.strip()])

    return questions[:min_questions]  # return only the top 3 questions

# generate questions from passages
for idx, passage in enumerate(passages):
    questions = generate_questions_pipeline(passage)
    print(f"Passage {idx+1}:\n{passage}\n")
    print("Generated Questions:")
    for q in questions:
        print(f"- {q}")
    print(f"\n{'-'*50}\n")

Device set to use cpu


In [44]:
# load the QA pipeline
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

# function to track and answer only unique questions
def answer_unique_questions(passages, qa_pipeline):
    answered_questions = set()  # to store unique questions

    for idx, passage in enumerate(passages):
        questions = generate_questions_pipeline(passage)

        for question in questions:
            if question not in answered_questions:  # check if the question has already been answered
                answer = qa_pipeline({'question': question, 'context': passage})
                print(f"Q: {question}")
                print(f"A: {answer['answer']}\n")
                answered_questions.add(question)  # add the question to the set to avoid repetition
        print(f"{'='*50}\n")

answer_unique_questions(passages, qa_pipeline)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu


In [27]:
import nltk.data

try:
    print(nltk.data.find('tokenizers/punkt'))
    print("✅ Punkt tokenizer is correctly installed!")
except LookupError:
    print("❌ Error: Punkt tokenizer not found!")



❌ Error: Punkt tokenizer not found!


In [24]:
# Step 1: Extract text from PDF using pdfplumber
import pdfplumber

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\
"
    return text

# Extract text from the PDF
pdf_text = extract_text_from_pdf('google_terms_of_service_en_in.pdf')

# Save the extracted text to a file for later use
with open('extracted_text.txt', 'w', encoding='utf-8') as f:
    f.write(pdf_text)

print("Text extracted and saved. First 500 characters:")
print(pdf_text[:500])

Text extracted and saved. First 500 characters:
GOOGLE TERMS OF SERVICE
Effective May 22, 2024 | Archived versions
What’s covered in these terms
We know it’s tempting to skip these Terms of
Service, but it’s important to establish what you
can expect from us as you use Google services,
and what we expect from you.
These Terms of Service re ect the way Google’s business works, the laws that apply to
our company, and certain things we’ve always believed to be true. As a result, these Terms
of Service help de ne Google’s relationship with you as


In [46]:
# Install required packages
%pip install transformers nltk

In [47]:
# Step 3: Summarize using T5
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

def generate_summary(text, max_chunk_length=512):
    # Initialize the model and tokenizer
    tokenizer = T5Tokenizer.from_pretrained('t5-small')
    model = T5ForConditionalGeneration.from_pretrained('t5-small')

    # Split text into chunks if it's too long
    chunks = [text[i:i+max_chunk_length] for i in range(0, len(text), max_chunk_length)]
    summaries = []

    for chunk in chunks:
        # Prepare the input
        inputs = tokenizer.encode("summarize: " + chunk, return_tensors="pt", max_length=512, truncation=True)

        # Generate summary
        summary_ids = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary)

    return " ".join(summaries)

# Read the extracted text
with open('extracted_text.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Generate and save summary
summary = generate_summary(text)
with open('summary.txt', 'w', encoding='utf-8') as f:
    f.write(summary)

print("Generated Summary:")
print(summary)

Generated Summary:
GOOGLE TERMS OF SERVICE Effective May 22, 2024 Archived versions What’s covered in these terms We know it’s tempting to skip these Terms of Service, but it’s important to establish what you can expect from us as you use Google services. these Terms of Service reect the way Google’s business works, the laws that apply to our company, and certain things we’ve always believed true. these terms include: what you can expect from us, which describes how we provide and develop our services What we expect from you, which establishes certain rules for using our services Content in Google services, which describes the intellectual property rights to the content you nd in our services. in case of problems or disagreements, which describes other legal rights you have. t to expect in case someone violates these terms Understanding these terms is important because, by accessing or using our services, you’re agreeing to these terms. besides these terms, we also publish a Privacy Po

In [49]:
import nltk
nltk.download('punkt_tab')



[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [50]:
import nltk
nltk.download('punkt_tab', download_dir='/path/to/nltk_data')


[nltk_data] Downloading package punkt_tab to /path/to/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [4]:
# Step 4: Split text into sentences using NLTK
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

def split_into_sentences(text):
    sentences = sent_tokenize(text)
    return sentences

# Read the text and split into sentences
with open('extracted_text.txt', 'r', encoding='utf-8') as f:
    text = f.read()

sentences = split_into_sentences(text)

# Save sentences to file
with open('sentences.txt', 'w', encoding='utf-8') as f:
    for sentence in sentences:
        f.write(sentence + '\
')

print("Number of sentences extracted:", len(sentences))
print("\
First 5 sentences:")
for i, sentence in enumerate(sentences[:5]):
    print(f"{i+1}. {sentence}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Number of sentences extracted: 126
First 5 sentences:
1. GOOGLE TERMS OF SERVICE
Effective May 22, 2024 | Archived versions
What’s covered in these terms
We know it’s tempting to skip these Terms of
Service, but it’s important to establish what you
can expect from us as you use Google services,
and what we expect from you.
2. These Terms of Service re ect the way Google’s business works, the laws that apply to
our company, and certain things we’ve always believed to be true.
3. As a result, these Terms
of Service help de ne Google’s relationship with you as you interact with our services.
4. For
example, these terms include the following topic headings:
What you can expect from us, which describes how we provide and develop our
services
What we expect from you, which establishes certain rules for using our services
Content in Google services, which describes the intellectual property rights to the
content you  nd in our services — whether that content belongs to you, Google, or
others


In [10]:
from transformers import pipeline

# Step 5: Generate questions using T5
def generate_questions(text):
    tokenizer = AutoTokenizer.from_pretrained("valhalla/t5-base-qg-hl")
    model = AutoModelForSeq2SeqLM.from_pretrained("valhalla/t5-base-qg-hl")

    max_length = 512
    questions = []

    input_text = " ".join(sentences[:10])

    inputs = tokenizer.encode("generate questions: " + input_text,
                            return_tensors="pt",
                            max_length=max_length,
                            truncation=True)

    outputs = model.generate(inputs,
                           max_length=64,
                           num_beams=5,
                           length_penalty=1.5,
                           num_return_sequences=5)

    for output in outputs:
        question = tokenizer.decode(output, skip_special_tokens=True)
        questions.append(question)

    return questions

# Generate questions
generated_questions = generate_questions(text)

# Save questions to file
with open('generated_questions.txt', 'w', encoding='utf-8') as f:
    for i, question in enumerate(generated_questions, 1):
        f.write(f"Q{i}: {question}\n")

print("\nGenerated Questions:")
for i, question in enumerate(generated_questions, 1):
    print(f"Q{i}: {question}")

# Step 6: Answer Generated Questions
qa_model = pipeline("question-answering", model="deepset/roberta-base-squad2")

def answer_question(question, context):
    answer = qa_model(question=question, context=context)
    return answer['answer']

print("\nAnswers to Generated Questions:")
for i, question in enumerate(generated_questions, 1):
    answer = answer_question(question, text)
    print(f"Q{i}: {question}")
    print(f"A: {answer}\n")


Generated Questions:
Q1: What is the name of the company that you are contracting with to use Google's services?
Q2: What is the name of the company that you are contracting with to use the services of Google?
Q3: What is the name of the company that you are contracting with to use Google services?
Q4: What is the name of the company that you are contracting with to use the Google services?
Q5: What is the name of the company that you are contracting with to use the services?


Device set to use cpu



Answers to Generated Questions:
Q1: What is the name of the company that you are contracting with to use Google's services?
A: Google LLC

Q2: What is the name of the company that you are contracting with to use the services of Google?
A: Google LLC

Q3: What is the name of the company that you are contracting with to use Google services?
A: Google LLC

Q4: What is the name of the company that you are contracting with to use the Google services?
A: Google LLC



KeyboardInterrupt: 

In [ ]:
import pdfplumber
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import nltk
import random

# Step 1: Extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

# Step 2: Split text into sentences
nltk.download('punkt')
def split_into_sentences(text):
    return nltk.sent_tokenize(text)

# Step 3: Generate diverse questions
def generate_diverse_questions(text, num_questions=10):
    tokenizer = AutoTokenizer.from_pretrained("t5-base")
    model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")

    sentences = split_into_sentences(text)
    questions = set()

    while len(questions) < num_questions:
        input_text = " ".join(random.sample(sentences, min(15, len(sentences))))

        inputs = tokenizer.encode("generate diverse questions: " + input_text,
                                return_tensors="pt",
                                max_length=512,
                                truncation=True)

        outputs = model.generate(inputs,
                               max_length=64,
                               num_beams=5,
                               length_penalty=1.5,
                               num_return_sequences=3,
                               temperature=0.8,
                               do_sample=True)

        for output in outputs:
            question = tokenizer.decode(output, skip_special_tokens=True)
            questions.add(question)

    return list(questions)[:num_questions]

# Step 4: Answer generated questions
qa_model = pipeline("question-answering", model="deepset/roberta-base-squad2")

def answer_question(question, context):
    answer = qa_model(question=question, context=context)
    return answer['answer']

# Main execution
pdf_path = "google_terms_of_service_en_in.pdf"
text = extract_text_from_pdf(pdf_path)

generated_questions = generate_diverse_questions(text)

print("\nGenerated Questions and Answers:")
for i, question in enumerate(generated_questions, 1):
    answer = answer_question(question, text)
    print(f"Q{i}: {question}")
    print(f"A: {answer}\n")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Device set to use cpu



Generated Questions and Answers:
Q1: Google agrees to personal jurisdiction in the federal or state courts of Santa Clara County, California, USA . if you’re a parent or legal guardian, and you allow your child to use the services, then these terms apply to you . license Your content remains yours, which
A: not a consumer



In [25]:
import pdfplumber
import nltk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch

# Download necessary NLTK data
nltk.download('punkt_tab')

# Step 1: Extract text from PDF using pdfplumber
import pdfplumber

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\
"
    return text

# Extract text from the PDF
pdf_text = extract_text_from_pdf('google_terms_of_service_en_in.pdf')

# Save the extracted text to a file for later use
with open('extracted_text.txt', 'w', encoding='utf-8') as f:
    f.write(pdf_text)

print("Text extracted and saved. First 500 characters:")
print(pdf_text[:500])


# Step 2: Preview Extracted Text
pdf_path = "your_document.pdf"  # Replace with your PDF file path
text = extract_text_from_pdf(pdf_path)
print("Extracted Text Preview:")
print(text[:500])  # Print first 500 characters

# Step 3: Summarize the Document
summarizer = pipeline("summarization", model="t5-small", tokenizer="t5-small", framework="pt")

def summarize_text(text, max_length=150, min_length=40):
    summary = summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)[0]['summary_text']
    return summary

summary = summarize_text(text)
print("\nDocument Summary:")
print(summary)

# Step 4: Split the Document
def split_into_sentences(text):
    return nltk.sent_tokenize(text)

sentences = split_into_sentences(text)

# Step 5: Generate diverse questions using T5
def generate_diverse_questions(text, num_questions=10):
    tokenizer = AutoTokenizer.from_pretrained("t5-base")
    model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")

    max_length = 512
    questions = set()  # Use a set to automatically remove duplicates

    input_text = " ".join(sentences[:15])  # Use more sentences for context

    inputs = tokenizer.encode("generate diverse questions: " + input_text,
                            return_tensors="pt",
                            max_length=max_length,
                            truncation=True)

    while len(questions) < num_questions:
        outputs = model.generate(inputs,
                               max_length=64,
                               num_beams=5,
                               length_penalty=1.5,
                               num_return_sequences=5,
                               temperature=0.8,  # Increase temperature for more diversity
                               do_sample=True)  # Enable sampling

        for output in outputs:
            question = tokenizer.decode(output, skip_special_tokens=True)
            questions.add(question)

    return list(questions)[:num_questions]

# Generate diverse questions
generated_questions = generate_diverse_questions(text)

# Save questions to file
with open('generated_questions.txt', 'w', encoding='utf-8') as f:
    for i, question in enumerate(generated_questions, 1):
        f.write(f"Q{i}: {question}\n")

print("\nGenerated Diverse Questions:")
for i, question in enumerate(generated_questions, 1):
    print(f"Q{i}: {question}")

# Step 6: Answer Generated Questions
qa_model = pipeline("question-answering", model="deepset/roberta-base-squad2")

def answer_question(question, context):
    answer = qa_model(question=question, context=context)
    return answer['answer']

print("\nAnswers to Generated Questions:")
for i, question in enumerate(generated_questions, 1):
    answer = answer_question(question, text)
    print(f"Q{i}: {question}")
    print(f"A: {answer}\n")


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Text extracted and saved. First 500 characters:
GOOGLE TERMS OF SERVICE
Effective May 22, 2024 | Archived versions
What’s covered in these terms
We know it’s tempting to skip these Terms of
Service, but it’s important to establish what you
can expect from us as you use Google services,
and what we expect from you.
These Terms of Service re ect the way Google’s business works, the laws that apply to
our company, and certain things we’ve always believed to be true. As a result, these Terms
of Service help de ne Google’s relationship with you as


FileNotFoundError: [Errno 2] No such file or directory: 'your_document.pdf'